In [1]:
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import cv2
import glob
import numpy as np
import os

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Retrieve image file paths from a specified folder
def get_image_paths(folder_path):
    return glob.glob(f'{folder_path}/**/*.jpg', recursive=True)

# Preprocess image by resizing and normalizing pixel values
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))
    return img

# Display both the original and processed images side by side
def display_original_and_processed(original, processed):
    plt.figure(figsize=(10,5))

    plt.subplot(1, 2, 1)
    plt.imshow(original)
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(processed)
    plt.title('Processed Image')

    plt.show()

# Process images from the specified folder and return the array of processed images
def process_images_from_folder(folder_path, images_to_display, total_images_to_process=None):
    image_paths = get_image_paths(folder_path)
    
    # If total_images_to_process is specified, limit the number of images to be processed
    if total_images_to_process:
        image_paths = image_paths[:total_images_to_process]
        
    processed_images = []
    
    for index, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        processed_img = preprocess_image(img)
        processed_images.append(processed_img)

        # Display images up to the number specified by images_to_display
        if index < images_to_display:
            display_original_and_processed(img, processed_img)

    return np.array(processed_images)

#face detection
def face_detection(pic):

    gpic = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
    facesdet = face_classifier.detectMultiScale(gpic, scaleFactor = 1.05, minNeighbors = 6, minSize=(25,25))
    return facesdet

#boxes
def box_disp(pic, facesdet, r1, r2):
    for (x, y, a, b) in facesdet:
        
        cv2.rectangle(pic, (x*r2, y*r1), (x*r2 + a*r2, y*r1 + b*r1), (0, 255, 0), 4)



# Function to detect faces in a still image and display it
def picCheck(image_path):
    image = cv2.imread(image_path)
    processed_image = preprocess_image(image)  # If needed, preprocess the image for face detection
    facesdet = face_detection(processed_image)

    # Calculate scaling factors (if the image was preprocessed)
    scaling_factor_y = image.shape[0] / processed_image.shape[0]
    scaling_factor_x = image.shape[1] / processed_image.shape[1]

    # Adjust face coordinates and dimensions based on the original image size
    adjusted_faces = [(int(x * scaling_factor_x), int(y * scaling_factor_y), int(a * scaling_factor_x), int(b * scaling_factor_y)) for x, y, a, b in facesdet]

    # Draw boxes on the original image
    box_disp(image, adjusted_faces, 1, 1)  # No additional scaling needed here

    cv2.namedWindow('Facial Recognition', cv2.WINDOW_NORMAL)
    cv2.imshow("Facial Recognition", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()




# Process video feed from a file or the webcam
def process_video(video_path=None):
    cap = cv2.VideoCapture(0 if video_path is None else video_path)

    if not cap.isOpened():
        print("Error: Couldn't open the video stream or file.")
        return
    
    window_name = 'Processed Video Stream'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    frame_counter = 0  
    face_counter = 0

    try:
        while cap.isOpened():
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:  
                break
            ret, frame = cap.read()
            if not ret:
                print("Error: Couldn't read a frame from the video.")
                break

            # Store original frame size
            original_height, original_width = frame.shape[:2]

            # Process the frame
            processed_frame = preprocess_image(frame)

            # Calculate scaling factors
            scaling_factor_y = original_height / processed_frame.shape[0]
            scaling_factor_x = original_width / processed_frame.shape[1]

            # Detect faces on the processed frame
            facesdet = face_detection(processed_frame)

            # Adjust face coordinates and dimensions based on the original frame size
            adjusted_faces = [(int(x * scaling_factor_x), int(y * scaling_factor_y), int(a * scaling_factor_x), int(b * scaling_factor_y)) for x, y, a, b in facesdet]

            # Draw boxes on the original frame
            box_disp(frame, adjusted_faces, 1, 1)  # No additional scaling needed here

            cv2.imshow(window_name, frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                frame_filename = os.path.join("C:\\Users\\mevin\\Desktop\\Untitled Folder 1\\ML--Age-Detection\\screenshots", f"screenshot_{frame_counter}.jpg")
                cv2.imwrite(frame_filename, frame) 
                frame_counter += 1
            elif key == ord('f'):  # Check if 'f' key is pressed
                for (x, y, w, h) in adjusted_faces:
                    # Crop the face from the frame
                    cropped_face = frame[y:y+h, x:x+w]

                    # Save the cropped face
                    face_filename = os.path.join("C:\\Users\\mevin\\Desktop\\Untitled Folder 1\\ML--Age-Detection\\faces", f"face_{face_counter}.jpg")
                    cv2.imwrite(face_filename, cropped_face)
                    face_counter += 1
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Functions to open a file dialog for selecting an image/video file
 
def select_picture():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        picCheck(file_path)

def select_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.mkv")])
    if file_path:
        process_video(file_path)


# Create the main window
window = tk.Tk()
window.title("Facial Recognition")
window.geometry("800x600")
window.configure(bg="#f0f0f0")

lbl_heading = tk.Label(window, text="Facial Recognition Application", font=("Arial", 24, "bold"), bg="#f0f0f0")
lbl_heading.pack(pady=20)

btn_select_picture = tk.Button(window, text="Select Picture", command=select_picture, font=("Arial", 14, "bold"),
                               bg="#4caf50", fg="white", padx=20, pady=10)
btn_select_picture.pack(pady=10)
btn_real_time_check = tk.Button(window, text="Select Video", command=select_video, font=("Arial", 14, "bold"),
                                bg="#4caf50", fg="white", padx=20, pady=10)
btn_real_time_check.pack(pady=10)
btn_real_time_check = tk.Button(window, text="Real-time Check", command=process_video, font=("Arial", 14, "bold"),
                                bg="#4caf50", fg="white", padx=20, pady=10)
btn_real_time_check.pack(pady=10)

menu_bar = tk.Menu(window)
window.config(menu=menu_bar)

file_menu = tk.Menu(menu_bar, tearoff=0)
file_menu.add_command(label="Select Picture", command=select_picture)
file_menu.add_command(label="Select Video", command=select_video)
file_menu.add_command(label="Real-time Check", command=process_video)
file_menu.add_command(label="Exit", command=window.quit)

menu_bar.add_cascade(label="File", menu=file_menu)

window.mainloop()